In [ ]:
from main import *
from models import *
from tqdm import tqdm

beta_1 = 1e-4
beta_T = 0.02
T = 500
shape = (44, 126, 16)
device = torch.device('cuda')

model = Model(device, beta_1, beta_T, T)
process = DiffusionProcess(beta_1, beta_T, T, model, device, shape)
optim = torch.optim.Adam(model.parameters(), lr = 0.0001)

total_iteration = 2000
current_iteration = 0

train_sub = [1,2,3,4,5,6,7,8]

train_x0, train_y0 = make_train_dataset(train_sub, 0)
train_stft = Stft_dataset(train_x0, train_y0)

dataloader = torch.utils.data.DataLoader(train_stft, batch_size = 32, drop_last = True, num_workers = 0)
dataiterator = iter(dataloader)

pbar = tqdm(range(total_iteration))


for epoch in pbar:
    losses = []
    num_items = 0
    
    for x, y in dataloader:
        data = x.to(device = device)
        loss = model.loss_fn(data)

        optim.zero_grad()
        loss.backward()
        optim.step()
        
        losses.append(loss.item())
        num_items += x.shape[0]
    pbar.set_description("Average loss : {}".format(sum(losses) / num_items))

In [ ]:
sampling_number = 4
only_final = True
process = DiffusionProcess(beta_1, beta_T, T, model, device, shape)
sample = process.sampling(sampling_number, only_final)



In [ ]:
from main import *

generated_signal = return_to_signal(sample)

In [ ]:

train_sub = [1]
val_sub = [9]
test_sub = [9]

main(train_sub, val_sub, test_sub)

In [ ]:
from main import *

device = "cuda"

train_sub = [1,2,3,4,5,6,7,8]
val_sub = [9]
test_sub = [9]

main(train_sub, val_sub, test_sub)
train_witout_aug(train_sub, val_sub, test_sub)

/home/junyeobe/.local/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
  0%|                                                                                                                                                                                                                               | 0/100 [00:00<?, ?it/s]/home/junyeobe/.local/lib/python3.9/site-packages/torch/nn/functional.py:3737: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
Average loss : 0.003951587067503069:  35%|█████████████████████████████████████████████████████████████▉                                                                              